In [1]:
# Imports
import csv, os
import pandas as pd

test_file_path = 'https://raw.githubusercontent.com/noharm-ai/substance-prediction/refs/heads/main/datasets/med_all_bio_test.csv'
test_file = pd.read_csv(test_file_path)

test_file

,medicamento,substancia
0,"Akynzeo (300mg + 0,56mg) Cáp.",NETUPITANTO + PALONOSETRONA
1,ACIDO ASCORBICO 1G EFERVESCENTE (VITAMINA C),ACIDO ASCORBICO (VITAMINA C)
2,Optiray 350 Sol inj 100mL (seringa),IOVERSOL
3,ACIDO ACETIL SALICILICO 100MG,ACIDO ACETILSALICILICO <200MG
4,Sulfato de Hidroxicloroquina 400 mg comp,HIDROXICLOROQUINA
...,...,...
1000,(Do paciente) Mesalazina 500 mg supositório,MESALAZINA
1001,Venlafaxina 150mg Cápsula De Liberação Prolon...,VENLAFAXINA
1002,"PIPERACILINA+TAZOBACTAM 4G+0,5G SOL.INJ",PIPERACILINA + TAZOBACTAM
1003,CLORETO DE POTASSIO 10% 10ML,CLORETO DE POTASSIO


## LLM - GPT-4

In [ ]:
# ChatGPT4
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint="[azure_endpoint]",
    api_version="[api_version]",
    api_key="[api_key]"
)

def get_gpt4_answer(message_text):

  completion = client.chat.completions.create(
      model="[model]",
      messages = message_text,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
  )

  return completion.choices[0].message.content

In [ ]:
# "Instruction: Based on the drug name provided, identify the most likely active substance."
# "Consider common variations such as prefixes, suffixes, or abbreviations, and disregard additional information"
# "such as dosages or pharmaceutical forms. Provide the answer with only the name of the active substance, without any additional text."

main_prompt = (
        "Instrução: Com base no nome do medicamento fornecido, identifique a substância ativa mais provável. "
        "Considere variações comuns, como prefixos, sufixos ou abreviações, e desconsidere informações adicionais, "
        "como dosagens ou formas farmacêuticas. Forneça a resposta apenas com o nome da substância ativa, sem nenhum texto adicional."
)

# Function to call the language model with the specific prompt
def query_llm(medicamento_nome):
    prompt = f"{main_prompt} Nome do medicamento: {medicamento_nome}"

    messages = [
      {"role":"system","content":"You are an AI assistant that helps people find information."},
      {"role":"user","content":prompt}
    ]

    # For ChatGPT
    response = get_gpt4_answer(messages)

    return response

# DataFrame to receive the correct drug and substance names
results = []

# Iterate over each drug and its substance
for _, row in test_file.iterrows():
    medicamento = row['medicamento']
    substancia_correta = row['substancia']

    # Consult LLM to obtain the expected substance
    llm_response = query_llm(medicamento)

    # Add the result to the list
    results.append({
        "medicamento": medicamento,
        "substancia_correta": substancia_correta,
        "llm_response": llm_response
    })

# Create the final DataFrame with the results
df_results = pd.DataFrame(results)

# Save to CSV
# For ChatGPT
df_results.to_csv("gpt_results.csv", index=False)
print("CSV file saved as 'gpt_results.csv'")

Arquivo CSV salvo como 'resultados_gpt.csv'


In [ ]:
import pandas as pd

# Function to create messages in LLM format
def criar_mensagem_linha(medicamento, substancia_correta, llm_response):
    return [
        {"role": "system", "content": "Você é um assistente especializado em validação de substâncias de medicamentos."},
        {"role": "user", "content": f"""
Analise os dados abaixo:
- Medicamento: {medicamento}
- Substância correta: {substancia_correta}
- Resposta da LLM: {llm_response}

Determine se a resposta da LLM é equivalente à substância correta, considerando variações nos nomes das substâncias (como ordem, maiúsculas/minúsculas, ou separadores).
Responda apenas com '1' para equivalente e '0' para não equivalente.
"""}
    ]

# Load the CSV
df = pd.read_csv('gpt_results.csv')

# Add a new column to the results (correct)
df['acertou'] = None

# Iterate through the rows and call LLM
for idx, row in df.iterrows():
    mensagem = criar_mensagem_linha(row['medicamento'], row['substancia_correta'], row['llm_response'])
    resposta = get_gpt4_answer(mensagem)
    try:
        # Convert the answer to 1 or 0 and store it in the 'acertou'(correct) column
        df.at[idx, 'acertou'] = int(resposta.strip())
    except ValueError:
        print(f"Error processing response for line {idx}: {resposta}")

# Calculates precision and recall
total_respostas = len(df)
total_acertos = df['acertou'].sum()
total_esperados = total_respostas

precision = total_acertos / total_respostas
recall = total_acertos / total_esperados

# Save the updated CSV
df.to_csv('evaluated_file.csv', index=False)

# Displays metrics
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")


Precisão: 92.84%
Recall: 92.84%


### Calculate amount of tokens

In [8]:
!pip install tiktoken
import tiktoken

# Load the appropriate model
encoding = tiktoken.encoding_for_model("gpt-4")

resultados_gpt = pd.read_csv('resultados_gpt.csv')

main_prompt = (
        "Instrução: Com base no nome do medicamento fornecido, identifique a substância ativa mais provável. "
        "Considere variações comuns, como prefixos, sufixos ou abreviações, e desconsidere informações adicionais, "
        "como dosagens ou formas farmacêuticas. Forneça a resposta apenas com o nome da substância ativa, sem nenhum texto adicional."
)

def get_prompt(medicamento_nome):
    prompt = f"{main_prompt} Nome do medicamento: {medicamento_nome}"

    messages = [
      {"role":"system","content":"You are an AI assistant that helps people find information."},
      {"role":"user","content":prompt}
    ]
    return messages

def get_answer(text):
    message = [
      {"role":"assistant","content":text}
    ]
    return message

def contar_tokens(mensage):
    tokens = sum(len(encoding.encode(msg["content"])) for msg in mensage)
    return tokens


counted_tokens = []
total_tokens = 0
total_input_tokens = 0
total_output_tokens = 0

for _, row in resultados_gpt.iterrows():
    medicamento = row['medicamento']
    llm_response = row['llm_response']


    input_tokens = contar_tokens(get_prompt(medicamento))
    output_tokens = contar_tokens(get_answer(llm_response))
    total_input_tokens += input_tokens
    total_output_tokens += output_tokens

    # Add the result to the list
    counted_tokens.append({
        "medicamento": medicamento,
        "llm_response": llm_response,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens
    })


counted_tokens.append({
        "medicamento": ' ',
        "llm_response": 'Total: ',
        "input_tokens": total_input_tokens,
        "output_tokens": total_output_tokens
    })
# Create the final DataFrame with the results
df_results = pd.DataFrame(counted_tokens)

# Save to CSV
df_results.to_csv("counted_tokens.csv", index=False)
print("CSV file saved as 'counted_tokens.csv'")


print(f"Total Input tokens: {total_input_tokens}")
print(f"Total Output tokens: {total_output_tokens}")
total_tokens = total_input_tokens + total_output_tokens
print(f"Total tokens: {total_tokens}")


CSV file saved as 'counted_tokens.csv'
Total Input tokens: 121636
Total Output tokens: 5986
Total tokens: 127622


### Results

**GPT 4**
*   Generation time: 3m23s
*   Evaluation time: 8m19s
*   Precision: 92.84%
*   Recall: 92.84%
*   Total Input tokens: 121636
*   Total Output tokens: 5986
*   Total tokens: 127622
